## General Description -

- This is the Python implementation of two proposed algorithms -
  -  CLOHusp - Closed High Utility Sequential Pattern Mining

  - RACLOHusp - Redundancy Aware Closed High Utility Sequential Pattern Mining

- Developed and implemented by -
  - Md Aminul Kader Bulbul
  - Reg. no - 2017814980
  - MS Student, CSEDU


In [ ]:
# Install gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install necessary libraries

In [ ]:
# Install necessary libraries

import math
import copy
import pandas as pd
import numpy as np
import itertools
import pickle
import time
import json
import os

Declare global variables

In [ ]:
# Declare global variables

df_chus = None

item_info = {}

unique_items = []
seq_util_val = []
util_list = []
seq_list = []

seq_no = {}
seq_pos_no = {}
seq_pos_no_util = {}
seq_pos_no_rem_util = {}
seq_pos_no_rbu = {}
seq_pos_no_lru = {}
seq_pos_no_rem_item_count = {}
item_seen_bef = [0] * 1570

item_neighbour = {}

temp_bitmap = []

keys_to_delete = []

item_info_dbv = {}

pattern_av_hash = {}

chus_hash = {}

visited = []
cluster_info = {}
cluster_info_per_reps = {}

Function implementation of Early Elimination by Support and SWU equivalence.


*   It checks whether pattern item_l is a sub-pattern or super-pattern of any other pattern in the database.
*   If it is a sub-pattern then merge it with super-pattern and stop further exploration.
*   Else if it is a super-pattern, then merge corresponding sub-patterns with it and stop further exploration.



In [ ]:
def pattern_avoidable(item_l):

  """
    Function implementation of Early Elimination by Support and SWU equivalence.
        - It checks whether pattern item_l is a sub-pattern or super-pattern of any other pattern in the database.
        - If it is a sub-pattern then merge it with super-pattern and stop further exploration.
        - Else if it is a super-pattern, then merge corresponding sub-patterns with it and stop further exploration.
  """

  _key_ = item_info_dbv[tuple(item_l)]['rem_item_count'] # set remaining item count as key of a dictionary
  swu_itemset_p = item_info_dbv[tuple(item_l)]['swu'] # Get the SWU value of item_l
  sup_itemset_p = item_info_dbv[tuple(item_l)]['support'] # Get the Support value of item_l


  if(_key_ not in pattern_av_hash): # If key is new
    pattern_av_hash[_key_] = [] # Declare an empty list against it

  mk = pattern_av_hash[_key_] # Get the list of patterns at key

  if(len(mk)==0): # if key has no pattern
    pattern_av_hash[_key_].append(item_l) # add item_l pattern to this key
    return False # Explore this pattern

  else:
    flag = 0
    for itemset_m in range(len(mk)): # for each pattern in the key
      if(itemset_m >= len(mk)): # if index not valid then break
        break
      itemset_p_bar = mk[itemset_m] # Get a pattern having same number of remaining itemset
      swu_itemset_p_bar = item_info_dbv[tuple(itemset_p_bar)]['swu'] # get its swu value
      sup_itemset_p_bar = item_info_dbv[tuple(itemset_p_bar)]['support'] # get its support value

      if(swu_itemset_p_bar == swu_itemset_p and
         sup_itemset_p_bar == sup_itemset_p): # If these 2 patterns have same SWU and Support value
        l_p = len(item_l)
        l_p_bar = len(itemset_p_bar)
        if(l_p <= l_p_bar and item_l[l_p-1] == itemset_p_bar[l_p_bar-1]): # if {item_l} is BACKWARD-SUB-PATTERN of {itemset_p_bar}
          return True # No need for further exploration
        else:
          if(l_p > l_p_bar and item_l[l_p-1] == itemset_p_bar[l_p_bar-1]): #if {item_l} is BACKWARD-SUPER-PATTERN of {itemset_p_bar}
            del pattern_av_hash[_key_][itemset_m] # then delete itemset_p_bar
            flag = 1 # mark that we entered into 2nd case

    if(flag==1): # if 2nd case
      pattern_av_hash[_key_].append(item_l) # add item_l into key
      return True

  pattern_av_hash[_key_].append(item_l) # add item_l into key
  return False # Explore this pattern

Function to explore the sibling patterns of item_l_child using DFS approach

In [ ]:
def exp_siblings(item_l_child, s_i_temp, i_i_temp, minsup, minutil):

  """
    Explore the sibling patterns of item_l_child - DFS approach
  """
  chus_cand_set_sib = []
  for item_l_child_i in range(len(item_l_child)): # For each pattern
    item_l_child_i_main = item_l_child[item_l_child_i]
    chus_cand_set_sib += dfs_pattern_ext(item_l_child_i_main, s_i_temp, i_i_temp[item_l_child_i:], minsup, minutil) # Explore it using DFS approach
  return chus_cand_set_sib

# Main function implementation of proposed CLOHusp algorithm
*   It explores the pattern item_l using DFS approach
*   It performs necessary operations on the dynamic vectors to calculate corresponding support, utility, remaining utility, SWU, RBU, LRU etc.
*   It performs both the S-Extension and I-extension
*   It uses LRU to identify which items are worthy of extension
*   It uses RBU to identify whether the pattern has potential to generate high-utility super patterns, else prune it
*   It returns the candidate CHUS patterns

In [ ]:
def dfs_pattern_ext(item_l,s_items,i_items,ms,mu):

  """
    Main function implementation of proposed CLOHusp algorithm
    - It explores the pattern item_l using DFS approach
    - It performs necessary operations on the dynamic vectors to calculate corresponding support, utility, remaining utility, SWU, RBU, LRU etc.
    - It performs both the S-Extension and I-extension
    - It uses LRU to identify which items are worthy of extension
    - It uses RBU to identify whether the pattern has potential to generate high-utility super patterns, else prune it
    - It returns the candidate CHUS patterns
  """

  if(not pattern_avoidable(item_l)): # if pattern item_l needs to be explored

    s_i_temp = []
    i_i_temp = []
    item_l_child = []
    chus_cand_set = []

    # S - Extension
    for sindx in range(len(s_items)): # for each s-items
      s_i = s_items[sindx]

      if((s_i not in item_neighbour[item_l[-1]]) or (s_i in item_l)): # if s-item is not neighbour to item_l
        continue # then no need to extend with it

      # Otherwise, explore with it
      # Get corresponding parameters
      givitem_sup_bmp_s = item_info_dbv[tuple(item_l)]['sup_bmp_s_sep']
      givitem_rbu_bmp = item_info_dbv[tuple(item_l)]['rbu_bmp_sep']
      sitem_sup_bmp = item_info_dbv[tuple([s_i])]['sup_bmp_sep']

      givitem_util_bmp_s = item_info_dbv[tuple(item_l)]['util_bmp_s_sep']
      sitem_util_bmp = item_info_dbv[tuple([s_i])]['util_bmp_sep']
      sitem_rem_util_bmp = item_info_dbv[tuple([s_i])]['rem_util_bmp_sep']

      sitem_swu_bmp = item_info_dbv[tuple([s_i])]['swu_bmp_sep']
      sitem_pos_rem_itc_bmp = item_info_dbv[tuple([s_i])]['pos_bmp_sep']

      giv_s_item_sup_bmp = []
      giv_s_item_lru_bmp = []
      giv_s_item_util_bmp = []
      giv_s_item_remutil_bmp = []
      giv_s_item_rbu_bmp = []
      giv_s_item_swu_bmp = []
      giv_s_item_pos_rem_itc_bmp = []

      giv_s_sup = 0
      giv_s_lru = 0
      giv_s_util = 0
      giv_s_seq_sum = 0
      giv_s_remutil = 0
      giv_s_swu = 0
      giv_s_rbu = 0
      giv_s_pos_rem_itc = 0

      giv_s_item_sup_bmp_s = []
      giv_s_item_util_bmp_s = []

      # Calculate corresponding support, utility, remaining utility, SWU, RBU, LRU etc.
      for zk in range(len(givitem_sup_bmp_s)):

        # Calculate support
        giv_s_item_sup_bmp_inner = [np.bitwise_and(a, b) for a, b in zip(givitem_sup_bmp_s[zk], sitem_sup_bmp[zk])]
        giv_s_item_sup_bmp.append(giv_s_item_sup_bmp_inner)
        sum_sup = __builtins__.sum(giv_s_item_sup_bmp_inner)
        giv_s_sup += sum_sup
        if(sum_sup!=0):
          giv_s_seq_sum += (zk+1)

        # Calculate LRU
        giv_s_item_lru_bmp_inner = [x * y for x, y in zip(givitem_rbu_bmp[zk], giv_s_item_sup_bmp_inner)]
        giv_s_item_lru_bmp.append(giv_s_item_lru_bmp_inner)
        giv_s_lru += __builtins__.sum(giv_s_item_lru_bmp_inner)

        # Calculate Utility
        giv_s_util_sum_bmp_inner = [__builtins__.sum(x) for x in zip(givitem_util_bmp_s[zk], sitem_util_bmp[zk])]
        giv_s_item_util_bmp_inner = [x * y for x, y in zip(giv_s_util_sum_bmp_inner, giv_s_item_sup_bmp_inner)]
        giv_s_item_util_bmp.append(giv_s_item_util_bmp_inner)
        giv_s_util += __builtins__.sum(giv_s_item_util_bmp_inner)

        # Calculate remaining utility
        giv_s_item_remutil_bmp_inner = [x * y for x, y in zip(sitem_rem_util_bmp[zk], giv_s_item_sup_bmp_inner)]
        giv_s_item_remutil_bmp.append(giv_s_item_remutil_bmp_inner)
        giv_s_remutil += __builtins__.sum(giv_s_item_remutil_bmp_inner)

        # Calculate RBU
        giv_s_item_rbu_bmp_inner = [__builtins__.sum(x) for x in
                                    zip(giv_s_item_util_bmp_inner, giv_s_item_remutil_bmp_inner)]
        sum_rbu = __builtins__.sum(giv_s_item_rbu_bmp_inner)
        giv_s_rbu += sum_rbu
        giv_s_item_rbu_bmp_inner = [sum_rbu] * len(giv_s_item_rbu_bmp_inner)
        giv_s_item_rbu_bmp.append(giv_s_item_rbu_bmp_inner)

        # Calculate SWU
        giv_s_item_swu_bmp_inner = [x * y for x, y in zip(sitem_swu_bmp[zk], giv_s_item_sup_bmp_inner)]
        giv_s_item_swu_bmp.append(giv_s_item_swu_bmp_inner)
        giv_s_swu += __builtins__.sum(giv_s_item_swu_bmp_inner)

        # Calculate remaining item count
        giv_s_item_pos_rem_itc_bmp_inner = [x * y for x, y in
                                            zip(sitem_pos_rem_itc_bmp[zk], giv_s_item_sup_bmp_inner)]
        giv_s_item_pos_rem_itc_bmp.append(giv_s_item_pos_rem_itc_bmp_inner)
        giv_s_pos_rem_itc += __builtins__.sum(giv_s_item_pos_rem_itc_bmp_inner)

        # Calculate S-bitmap and S-Util-bitmap
        first_nonzero_index = next((i for i, x in enumerate(giv_s_item_sup_bmp_inner) if x != 0), None)
        if first_nonzero_index is not None:
              _util_ = giv_s_item_util_bmp_inner[first_nonzero_index]
              giv_s_item_sup_bmp_s.append([0]*(first_nonzero_index+1) + [1]*(len(giv_s_item_sup_bmp_inner)-first_nonzero_index-1))
              giv_s_item_util_bmp_s.append([0]*(first_nonzero_index+1) + [_util_]*(len(giv_s_item_sup_bmp_inner)-first_nonzero_index-1))
        else:
              giv_s_item_sup_bmp_s.append([0]*len(giv_s_item_sup_bmp_inner))
              giv_s_item_util_bmp_s.append([0]*len(giv_s_item_sup_bmp_inner))


      # Check which items are worthy of extension
      if(giv_s_sup>=ms and giv_s_lru>=mu):
        s_i_temp.append(s_i)

        itl_org_cps = copy.deepcopy(item_l)
        itl_org_cps.append(s_i)

        # Insert the S-extended new pattern info into global dictionary
        item_info_dbv[tuple(itl_org_cps)] = {

                         'sup_bmp_sep': giv_s_item_sup_bmp, #--->>> ok
                         'util_bmp_sep': giv_s_item_util_bmp, #--->>> ok
                         'rem_util_bmp_sep': giv_s_item_remutil_bmp, #--->>> ok
                         'rbu_bmp_sep': giv_s_item_rbu_bmp, #--->>> ok
                         'lru_bmp_sep': giv_s_item_lru_bmp, #--->>> ok
                         'swu_bmp_sep': giv_s_item_swu_bmp, #--->>> ok
                         'pos_bmp_sep': giv_s_item_pos_rem_itc_bmp, #--->>> ok
                         'sup_bmp_s_sep': giv_s_item_sup_bmp_s, #--->>> ok
                         'util_bmp_s_sep': giv_s_item_util_bmp_s, #--->>> ok

                         'support': giv_s_sup,
                         'utility': giv_s_util,
                         'seq_sum': giv_s_seq_sum,
                         'rem_utils': giv_s_remutil,
                         'swu': giv_s_swu,
                         'rbu': giv_s_rbu,
                         'lru': giv_s_lru,
                         'rem_item_count': giv_s_pos_rem_itc

                         }

    # Check whether the S-extended new pattern has potential to generate high-utility super-patterns
    for sindx_tmp in range(len(s_i_temp)):
      s_i_main = s_i_temp[sindx_tmp]
      gs_item = item_l + [s_i_main]
      gs_rbu = item_info_dbv[tuple(gs_item)]['rbu']
      if(gs_rbu>=mu):
        item_l_child.append(gs_item) # if RBU > MinUtil, then extendable
      else:
        del item_info_dbv[tuple(gs_item)] # otherwise not

    chus_cand_set += item_l_child # Get all the child patterns of item_l
    chus_cand_set += exp_siblings(item_l_child, s_i_temp, s_i_temp, ms, mu) # Explore their siblings

    # I - Extension
    for iindx in range(len(i_items)): # for each s-items
      i_i = i_items[iindx]

      if((i_i not in item_neighbour[item_l[-1]]) or (i_i in item_l)): # if i-item is not neighbour to item_l
        continue # then no need to extend with it

      # Otherwise, explore with it
      # Get corresponding parameters
      givitem_sup_bmp_s = item_info_dbv[tuple(item_l)]['sup_bmp_sep'] # Modified
      givitem_rbu_bmp = item_info_dbv[tuple(item_l)]['rbu_bmp_sep']
      sitem_sup_bmp = item_info_dbv[tuple([i_i])]['sup_bmp_sep']

      givitem_util_bmp_s = item_info_dbv[tuple(item_l)]['util_bmp_sep'] # Modified
      sitem_util_bmp = item_info_dbv[tuple([i_i])]['util_bmp_sep']
      sitem_rem_util_bmp = item_info_dbv[tuple([i_i])]['rem_util_bmp_sep']

      sitem_swu_bmp = item_info_dbv[tuple([i_i])]['swu_bmp_sep']
      sitem_pos_rem_itc_bmp = item_info_dbv[tuple([i_i])]['pos_bmp_sep']

      giv_s_item_sup_bmp = []
      giv_s_item_lru_bmp = []
      giv_s_item_util_bmp = []
      giv_s_item_remutil_bmp = []
      giv_s_item_rbu_bmp = []
      giv_s_item_swu_bmp = []
      giv_s_item_pos_rem_itc_bmp = []

      giv_s_sup = 0
      giv_s_lru = 0
      giv_s_util = 0
      giv_s_seq_sum = 0
      giv_s_remutil = 0
      giv_s_swu = 0
      giv_s_rbu = 0
      giv_s_pos_rem_itc = 0

      giv_s_item_sup_bmp_s = []
      giv_s_item_util_bmp_s = []

      # Calculate corresponding support, utility, remaining utility, SWU, RBU, LRU etc.
      for zk in range(len(givitem_sup_bmp_s)):

        # Calculate support
        giv_s_item_sup_bmp_inner = [np.bitwise_and(a, b) for a, b in zip(givitem_sup_bmp_s[zk], sitem_sup_bmp[zk])]
        giv_s_item_sup_bmp.append(giv_s_item_sup_bmp_inner)
        sum_sup = __builtins__.sum(giv_s_item_sup_bmp_inner)
        giv_s_sup += sum_sup
        if(sum_sup!=0):
          giv_s_seq_sum += (zk+1)

        # Calculate LRU
        giv_s_item_lru_bmp_inner = [x * y for x, y in zip(givitem_rbu_bmp[zk], giv_s_item_sup_bmp_inner)]
        giv_s_item_lru_bmp.append(giv_s_item_lru_bmp_inner)
        giv_s_lru += __builtins__.sum(giv_s_item_lru_bmp_inner)

        # Calculate Utility
        giv_s_util_sum_bmp_inner = [__builtins__.sum(x) for x in zip(givitem_util_bmp_s[zk], sitem_util_bmp[zk])]
        giv_s_item_util_bmp_inner = [x * y for x, y in zip(giv_s_util_sum_bmp_inner, giv_s_item_sup_bmp_inner)]
        giv_s_item_util_bmp.append(giv_s_item_util_bmp_inner)
        giv_s_util += __builtins__.sum(giv_s_item_util_bmp_inner)

        # Calculate remaining utility
        giv_s_item_remutil_bmp_inner = [x * y for x, y in zip(sitem_rem_util_bmp[zk], giv_s_item_sup_bmp_inner)]
        giv_s_item_remutil_bmp.append(giv_s_item_remutil_bmp_inner)
        giv_s_remutil += __builtins__.sum(giv_s_item_remutil_bmp_inner)

        # Calculate RBU
        giv_s_item_rbu_bmp_inner = [__builtins__.sum(x) for x in
                                    zip(giv_s_item_util_bmp_inner, giv_s_item_remutil_bmp_inner)]
        sum_rbu = __builtins__.sum(giv_s_item_rbu_bmp_inner)
        giv_s_rbu += sum_rbu
        giv_s_item_rbu_bmp_inner = [sum_rbu] * len(giv_s_item_rbu_bmp_inner)
        giv_s_item_rbu_bmp.append(giv_s_item_rbu_bmp_inner)

        # Calculate SWU
        giv_s_item_swu_bmp_inner = [x * y for x, y in zip(sitem_swu_bmp[zk], giv_s_item_sup_bmp_inner)]
        giv_s_item_swu_bmp.append(giv_s_item_swu_bmp_inner)
        giv_s_swu += __builtins__.sum(giv_s_item_swu_bmp_inner)

        # Calculate remaining item count
        giv_s_item_pos_rem_itc_bmp_inner = [x * y for x, y in
                                            zip(sitem_pos_rem_itc_bmp[zk], giv_s_item_sup_bmp_inner)]
        giv_s_item_pos_rem_itc_bmp.append(giv_s_item_pos_rem_itc_bmp_inner)
        giv_s_pos_rem_itc += __builtins__.sum(giv_s_item_pos_rem_itc_bmp_inner)


      # Check which items are worthy of extension
      if(giv_s_sup>=ms and giv_s_lru>=mu):
        i_i_temp.append(i_i)

        itl_org_cps = copy.deepcopy(item_l)
        itl_org_cps.append(i_i)

        # Insert the S-extended new pattern info into global dictionary
        item_info_dbv[tuple(itl_org_cps)] = {

                         'sup_bmp_sep': giv_s_item_sup_bmp, #--->>> ok
                         'util_bmp_sep': giv_s_item_util_bmp, #--->>> ok
                         'rem_util_bmp_sep': giv_s_item_remutil_bmp, #--->>> ok
                         'rbu_bmp_sep': giv_s_item_rbu_bmp, #--->>> ok
                         'lru_bmp_sep': giv_s_item_lru_bmp, #--->>> ok
                         'swu_bmp_sep': giv_s_item_swu_bmp, #--->>> ok
                         'pos_bmp_sep': giv_s_item_pos_rem_itc_bmp, #--->>> ok
                         'sup_bmp_s_sep': giv_s_item_sup_bmp_s, #--->>> ok
                         'util_bmp_s_sep': giv_s_item_util_bmp_s, #--->>> ok

                         'support': giv_s_sup,
                         'utility': giv_s_util,
                         'seq_sum': giv_s_seq_sum,
                         'rem_utils': giv_s_remutil,
                         'swu': giv_s_swu,
                         'rbu': giv_s_rbu,
                         'lru': giv_s_lru,
                         'rem_item_count': giv_s_pos_rem_itc

                         }

    # Check whether the I-extended new pattern has potential to generate high-utility super-patterns
    for sindx_tmp in range(len(s_i_temp)):
      i_i_main = i_i_temp[sindx_tmp]
      gs_item = item_l + [i_i_main]
      gs_rbu = item_info_dbv[tuple(gs_item)]['rbu']
      if(gs_rbu>=mu):
        item_l_child.append(gs_item) # if RBU > MinUtil, then extendable
      else:
        del item_info_dbv[tuple(gs_item)] # otherwise not


    chus_cand_set += item_l_child # Get all the child patterns of item_l
    chus_cand_set += exp_siblings(item_l_child, s_i_temp, i_i_temp, ms, mu) # Explore their siblings

    return chus_cand_set # Return the candidate CHUS set

  else: # if pattern item_l avoidable
    return [] # then return empty list


Checks if a sub-list is contained within a super-list

In [ ]:
def is_sublist(sub_list, super_list):
  """Checks if a sub-list is contained within a super-list"""
  return all(x in super_list for x in sub_list)

Function to eliminate non-closed patterns from the candidate CHUS set using 2-level hash function

In [ ]:
def elim_non_closed(cand_set, minutl):

  """
    Eliminate non-closed patterns from the candidate CHUS set using 2-level hash function
  """
  chus_set = []
  key_found_bef = {}
  for ik in range(len(cand_set)): # For each candidate
    if isinstance(cand_set[ik], int):
       its = cand_set[ik]
       itl = []
       itl.append(its)
       item_l = itl
    else:
      item_l = cand_set[ik]
    if(item_info_dbv[tuple(item_l)]['utility']>=minutl): # if it is a high-utility pattern
      key = item_info_dbv[tuple(item_l)]['seq_sum'] # Get the summation of sequence numbers where it occured
      # Create a hash-entry with the summation of sequence numbers as key and item_l as value
      if(key not in key_found_bef):
        chus_hash[key] = []
        chus_hash[key].append(item_l)
        key_found_bef[key] = 1
      else:
        chus_hash[key].append(item_l)
        key_found_bef[key] += 1

  for key, value in chus_hash.items(): # for each hash entry

    val1 = copy.deepcopy(value)
    val2 = copy.deepcopy(value)
    for i1 in val1:
      f=0
      for i2 in reversed(val2):
        # if pattern i1 and i2 has same support
        if((i1 != i2) and (item_info_dbv[tuple(i1)]['support'] == item_info_dbv[tuple(i2)]['support'])):
          if(is_sublist(i1,i2)): # if pattern i1 is a sublist of i2
            f=1
            break # i1 can't be a CHUS and so no need to check further
          elif(is_sublist(i2,i1)): # if pattern i2 is a sublist of i1
            indx = val2.index(i2)
            # delete i2 as i2 can't be a CHUS
            del val2[indx]
            del val1[indx]

      if(f==0):
        chus_set.append(i1) # if i1 is a CHUS, then append it to chus_set

  return chus_set

Function to calculate the pattern distance between pr and p

In [ ]:
def calculate_pattern_dist(pr,p):

  """
    Calculate the pattern distance between pr and p
  """

  pr_sup = item_info_dbv[tuple(pr)]['support']
  pr_util = item_info_dbv[tuple(pr)]['utility']

  p_sup = item_info_dbv[tuple(p)]['support']
  p_util = item_info_dbv[tuple(p)]['utility']

  common_elements = list(set(pr) & set(p))

  sup_dist = 1 - (pr_sup/p_sup) # Calculate Support distance
  patt_dist = 1 - (len(common_elements)/len(p)) # Calculate Pattern similarity
  util_dist = (abs(pr_util-p_util))/(max(pr_util,p_util)) # Calculate Utility distance

  ov_dist = (sup_dist * 2) + (patt_dist * 6) + (util_dist * 2) # Calculate Overall Pattern Distance

  return ov_dist

Function to find neighbours within Redundancy Limit Threshold (Eps)


In [ ]:
def findNeighbours(D,P,eps):

    """
      Function to find neighbours within Redundancy Limit Threshold (Eps)
    """

    neighbours=[] ##empty list of indices
    for pnt in range(0,len(D)):
        if pnt!=P:
            if ((calculate_pattern_dist(D[P],D[pnt])) < eps): # Pattern distance is less than eps
                neighbours.append(pnt) # add neighbour

    return neighbours

# Proposed RACLOHusp algorithm to perform density based pattern clustering

## Algorithm Here

### Input:
 - chus : a set containing CHUS patterns
 - eps : the radius parameter, **(Redundancy Limit Threshold)**
 - MinPts : the neighborhood density threshold **(Minimum number of CHUS patterns which will be represented by the Core pattern of the cluster)**

### Output: A set of density based clusters.

### Method

mark all objects as unvisited;
- do
    - randomly select an unvisited object p;
    - mark p as visited;
    - if the eps-neighborhood of p has at least MinPts objects
        - create a new cluster C, and add p to C;
        - let N be the set of objects in the eps-neighborhood of p;
        - for each point p' in N
            - if p' is unvisited
                - mark p' as visited;
                - if the eps-neighborhood of p' has at least MinPts points
                - add those points to N;
            - if p' is not yet a member of any cluster, add p' to C;
        - end for
        - output C;
    - else mark p as noise;
- until no object is unvisited;

---------------------------------------------------------------------------------------------------
I am following the DBSCAN format of Data Mining Concepts and Techniques by Jiawei Han, Micheline Kamber and Jian Pei
   



In [ ]:
def clustering(_chus_,eps,MinPts):

  """
    Proposed RACLOHusp algorithm to perform density based pattern clustering
  """
  cluster_id = 0
  labels = [0]*len(_chus_)
  rep_patterns = []
  _chus_.sort(key=len)
  for i in reversed(range(len(_chus_))):

    if i not in visited:
      visited.append(i) ## mark as visited
      rep_patterns.append(_chus_[i])
      cluster_info_per_reps[tuple(_chus_[i])] = []
      neighbourPts=findNeighbours(_chus_,i,eps)
      if len(neighbourPts)<MinPts:
          labels[i]=-1 ## label point as noise/outlier
          cluster_info[tuple(_chus_[i])]=-1
      else:
          labels[i]=cluster_id
          cluster_info[tuple(_chus_[i])]=cluster_id
          ## expand cluster
          for p in neighbourPts:
              if p not in visited:
                 visited.append(p) ## mark as visited
                 pneighbourPts=findNeighbours(_chus_,p,eps)

                 ## if p in branch point add it's neightbours in neighbourPts
                 if len(pneighbourPts)>=MinPts:
                      for pnt in pneighbourPts:
                          if pnt not in neighbourPts:
                              neighbourPts.append(pnt) ## adding patterns to neighbourhood
              ## if p not in any cluster
              if labels[p]==0:
                  labels[p]=cluster_id
                  cluster_info[tuple(_chus_[p])]=cluster_id
                  cluster_info_per_reps[tuple(_chus_[i])].append(_chus_[p])

          cluster_id+=1 ## create a new cluster

  return labels,rep_patterns

## Main function -
*    It performs dataset scan
*    It removes low utility and low frequent patterns
*    It creates dynamic vectors like support, utility, remaining utility, SWU, RBU, LRU etc. of items
*    It calls the CLOHusp and RACLOHusp algorithms to generate CHUS patterns and perform density based pattern clustering to generate RACHUS patterns.

In [ ]:
def main():

  """
    Main function -
      - It performs dataset scan
      - It removes low utility and low frequent patterns
      - It creates dynamic vectors like support, utility, remaining utility, SWU, RBU, LRU etc. of items
      - It calls the CLOHusp and RACLOHusp algorithms to generate CHUS patterns and perform density based pattern clustering to generate RACHUS patterns.
  """

  filename = '/content/drive/MyDrive/MSThesis/CHUS-4/Dataset/Mushroom.txt' # Read dataset from drive like, Mushroom

  file = open(filename, 'r')

  unique_item_set = set()


  # Initialize some free lists
  item_list_rep_sup = [0] * 1570
  item_list_rep_util = [0] * 1570
  item_list_rep_remutil = [0] * 1570
  item_list_rep_remicount = [0] * 1570
  item_list_rep_total_swu = [0] * 1570
  item_list_rep_total_rbu = [0] * 1570
  item_seq_sum = [0] * 1570

  sp=0
  ut=0
  rm=0
  seq_sum = 0
  tran_util=0
  item_cswu=0
  item_crbu=0
  rm_it_cnt=0
  i=0
  total_util = 0

  for line in file: # Process each sequences

    row = [int(x) for x in line.replace(":", " ").split()]

    m = math.floor(len(row)/2)
    seq_util_val.append(row[m]) # Get Sequence utility
    tran_util = row[m]
    total_util += row[m]
    ul = row[m+1:]
    sl = row[:m]
    temp_set = set(sl)
    unique_item_set.update(temp_set)

    temp_bitmap_inner = [0] * len(sl)
    temp_bitmap.append(temp_bitmap_inner)

    for z in range(len(sl)):

        # Calculate Support
        item_list_rep_sup[sl[z]-1] += 1
        sp = item_list_rep_sup[sl[z]-1]

        # Calculate Utility
        item_list_rep_util[sl[z]-1] += ul[z]
        ut = item_list_rep_util[sl[z]-1]

        # Calculate Summation of sequences
        item_seq_sum[sl[z]-1] += (i+1)
        seq_sum = item_seq_sum[sl[z]-1]

        # Calculate Remaining utility
        item_list_rep_remutil[sl[z]-1] += __builtins__.sum(ul[(z+1):])
        rm = item_list_rep_remutil[sl[z]-1]

        # Calculate SWU
        item_list_rep_total_swu[sl[z]-1] += tran_util
        item_cswu = item_list_rep_total_swu[sl[z]-1]

        # Calculate RBU
        item_list_rep_total_rbu[sl[z]-1] += __builtins__.sum(ul[z:])
        item_crbu = item_list_rep_total_rbu[sl[z]-1]

        # Calculate Remaining Item Count
        item_list_rep_remicount[sl[z]-1] += len(sl)-(z+1)
        rm_it_cnt = item_list_rep_remicount[sl[z]-1]

        if(item_seen_bef[sl[z]]==0):

          item_neighbour[sl[z]] = set()
          item_neighbour[sl[z]].update(set(sl[z+1:])) # add adjacent items

          seq_no[sl[z]] = []
          seq_no[sl[z]].append(i)

          seq_pos_no[sl[z]] = []
          seq_pos_no[sl[z]].append(z)

          seq_pos_no_util[sl[z]] = []
          seq_pos_no_util[sl[z]].append(ul[z])

          seq_pos_no_rem_util[sl[z]] = []
          seq_pos_no_rem_util[sl[z]].append((__builtins__.sum(ul[(z+1):])))

          seq_pos_no_rbu[sl[z]] = []
          seq_pos_no_rbu[sl[z]].append((__builtins__.sum(ul[z:])))

          seq_pos_no_lru[sl[z]] = []
          seq_pos_no_lru[sl[z]].append(tran_util)

          seq_pos_no_rem_item_count[sl[z]] = []
          seq_pos_no_rem_item_count[sl[z]].append(len(sl)-(z+1))

          item_seen_bef[sl[z]] = 1

        else:
          item_neighbour[sl[z]].update(set(sl[z+1:]))
          seq_no[sl[z]].append(i)
          seq_pos_no[sl[z]].append(z)
          seq_pos_no_util[sl[z]].append(ul[z])
          seq_pos_no_rem_util[sl[z]].append(__builtins__.sum(ul[(z+1):]))
          seq_pos_no_rbu[sl[z]].append(__builtins__.sum(ul[z:]))
          seq_pos_no_lru[sl[z]].append(tran_util)
          seq_pos_no_rem_item_count[sl[z]].append(len(sl)-(z+1))

        item_info[sl[z]] = {'support': sp,
                            'utility': ut,
                            'seq_sum': seq_sum,
                            'rem_util': rm,
                            'swu': item_cswu,
                            'rbu': item_crbu,
                            'lru': item_cswu,
                            'rem_ic': rm_it_cnt}

    util_list.append(ul)
    seq_list.append(sl)
    i+=1
  file.close()
  unique_items = list(unique_item_set)


  mu_perc = 0.06 # Min Utility in percentage
  minsup = 4874 # Min Support
  minutil = int(total_util*mu_perc)

  print("-----------General Description--------------")
  print(f"Dataset: Mushroom")
  print(f"Total no of transactions: {i}")
  print(f"Total no of items: {len(unique_items)}")
  print(f"Total Util: {total_util}")
  print(f"ms: {minsup}")
  print(f"mu: {minutil}")
  print(f"mu-perc: {mu_perc}")


  rem_items = []

  # Identify the frequent-1 items having high-utility
  for key, value in item_info.items():
    if(value['support']<minsup or value['swu']<minutil):
      keys_to_delete.append(key)
    else:
      rem_items.append(key)

  for key in keys_to_delete:
    del item_info[key]

  print(f"Total no of remaining items: {len(rem_items)}")
  print("--------------------------------------------")
  print()


  # Create bitmaps of support, utility, remaining utility, SWU, RBU, LRU etc. of items
  for item_j in range (len(rem_items)):

    _item_ = rem_items[item_j]

    ib = copy.deepcopy(temp_bitmap)
    ibs = copy.deepcopy(temp_bitmap)

    iub = copy.deepcopy(temp_bitmap)
    iubs = copy.deepcopy(temp_bitmap)

    irub = copy.deepcopy(temp_bitmap)
    ipb = copy.deepcopy(temp_bitmap)

    i_rbu = copy.deepcopy(temp_bitmap)
    i_lru = copy.deepcopy(temp_bitmap)

    for iter in range(len(seq_no[_item_])):

      seq_no_i = seq_no[_item_][iter]
      seq_pos_no_i = seq_pos_no[_item_][iter]
      seq_pos_no_util_i = seq_pos_no_util[_item_][iter]
      seq_pos_no_rem_util_i = seq_pos_no_rem_util[_item_][iter]
      seq_pos_no_rbu_i = seq_pos_no_rbu[_item_][iter]
      seq_pos_no_lru_i = seq_pos_no_lru[_item_][iter]
      seq_pos_no_rem_item_count_i = seq_pos_no_rem_item_count[_item_][iter]

      seq_pos_no_next = seq_pos_no_i + 1

      ib[seq_no_i][seq_pos_no_i] = 1
      tmp_list = copy.deepcopy(ibs[seq_no_i])
      tmp_list_a = tmp_list[:seq_pos_no_next]
      tmp_list_b = tmp_list[seq_pos_no_next:]
      tmp_list_b = [1]*len(tmp_list_b)
      ibs[seq_no_i] = tmp_list_a + tmp_list_b

      iub[seq_no_i][seq_pos_no_i] = seq_pos_no_util_i
      tmp_list = copy.deepcopy(iubs[seq_no_i])
      tmp_list_a = tmp_list[:seq_pos_no_next]
      tmp_list_b = tmp_list[seq_pos_no_next:]
      tmp_list_b = [seq_pos_no_util_i]*len(tmp_list_b)
      iubs[seq_no_i] = tmp_list_a + tmp_list_b

      irub[seq_no_i][seq_pos_no_i] = seq_pos_no_rem_util_i
      ipb[seq_no_i][seq_pos_no_i] = seq_pos_no_rem_item_count_i

      len_i_rbu = len(i_rbu[seq_no_i])
      tmp_i_rbu = [seq_pos_no_rbu_i]*len_i_rbu
      i_rbu[seq_no_i] = tmp_i_rbu


      i_lru[seq_no_i][seq_pos_no_i] = seq_pos_no_lru_i

    item_info_dbv[tuple([_item_])] = {

                         'sup_bmp_sep': ib, #--->>> ok
                         'util_bmp_sep': iub, #--->>> ok
                         'rem_util_bmp_sep': irub, #--->>> ok
                         'rbu_bmp_sep': i_rbu, #--->>> ok
                         'lru_bmp_sep': i_lru, #--->>> ok
                         'swu_bmp_sep': i_lru, #--->>> ok
                         'pos_bmp_sep': ipb, #--->>> ok
                         'sup_bmp_s_sep': ibs, #--->>> ok
                         'util_bmp_s_sep': iubs, #--->>> ok

                         'support': item_info[_item_]['support'],
                         'utility': item_info[_item_]['utility'],
                         'seq_sum': item_info[_item_]['seq_sum'],
                         'rem_utils': item_info[_item_]['rem_util'],
                         'swu': item_info[_item_]['swu'],
                         'rbu': item_info[_item_]['rbu'],
                         'lru': item_info[_item_]['lru'],
                         'rem_item_count': item_info[_item_]['rem_ic']

                         }

  chus_cand = []

  chus_cand += rem_items

  for item_i in range(len(rem_items)): # For each high utility items

    # Call the CLOHusp algorithm to generate candidate CHUS pattrns
    chus_cand += dfs_pattern_ext([rem_items[item_i]],rem_items,rem_items[(item_i+1):],minsup,minutil)

  # Create a Data Frame for Cand-CHUS
  sc=[]
  uc=[]
  print()
  print(f"Total no of Cand. CHUS: {len(chus_cand)}")
  for i in range(len(chus_cand)):
    if(isinstance(chus_cand[i], int)):
      ck = [chus_cand[i]]
    else:
      ck = chus_cand[i]
    sc += [item_info_dbv[tuple(ck)]['support']]
    uc += [item_info_dbv[tuple(ck)]['utility']]
  df_chusc = pd.DataFrame({'CHUS Candidate - Patterns': chus_cand, 'Support': sc, 'Utility': uc})

  chus_set = elim_non_closed(chus_cand,minutil) # Eliminate the non-closed candidates to generate the CHUS patterns

  print(f"Total no of CHUS: {len(chus_set)}")
  print()

  sups = [item_info_dbv[tuple(key)]['support'] for key in chus_set if tuple(key) in item_info_dbv]
  uts  = [item_info_dbv[tuple(key)]['utility'] for key in chus_set if tuple(key) in item_info_dbv]

  # Call the RACLOHusp algorithm to generate RACHUS pattrns by density based pattern clustering
  # Here, eps = 0.73 and MinPatts = 2
  labs,reps = clustering(chus_set,0.73,2)

  # Create a Data Frame for RACHUS
  sups_r = [item_info_dbv[tuple(key)]['support'] for key in reps if tuple(key) in item_info_dbv]
  uts_r  = [item_info_dbv[tuple(key)]['utility'] for key in reps if tuple(key) in item_info_dbv]
  lbls_r = [cluster_info[tuple(key)] for key in reps if tuple(key) in item_info_dbv]
  df_chus_r = pd.DataFrame({'CHUS - Patterns': reps, 'Support': sups_r, 'Utility': uts_r, 'Cluster ID': lbls_r})

  # Create a Data Frame for CHUS
  lbls = [cluster_info[tuple(key)] for key in chus_set if tuple(key) in item_info_dbv]
  df_chus = pd.DataFrame({'CHUS - Patterns': chus_set, 'Support': sups, 'Utility': uts, 'Cluster ID': lbls})

  print(f"Total no of Rep-CHUS: {len(reps)}")
  print()

  return df_chus, chus_set, df_chus_r, reps, labs # Return corresponding outputs


In [ ]:
%%time
df, _chus_, dfr, rps, lbs = main()